In [ ]:
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense, LSTM

In [ ]:
sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")

In [ ]:
stock = "AAPL"

In [ ]:
end = datetime.now()
start = datetime(end.year - 10, end.month, end.day)

In [ ]:
df = yf.download(stock, start, end)
df

In [ ]:
plt.figure(figsize=(16,9))
sns.lineplot(x="Date", y="Close", data=df)

In [ ]:
data = df.filter(['Close'])
data

In [ ]:
# # replace Close with the count of days
# data["Close"] = range(1, len(data) + 1)
# data

In [ ]:
dataset = data.values
dataset

In [ ]:
# Scale the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

scaled_data, max(scaled_data), min(scaled_data)

In [ ]:
# train_data = scaled_data[0:int(training_data_len), :]
# test_data = scaled_data[training_data_len - 60:, :]

# train_data

In [ ]:
# len(train_data), len(test_data)

In [ ]:
# How many days to look back
days = 60

x = []
y = []

# Create two arrays, x_train and y_train
# x_train contain arrays of 60 days of data
# y_train contain the 61st day data
for i in range(days, len(scaled_data)):
    x.append(scaled_data[i-days:i, 0])
    y.append(scaled_data[i, 0])

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.05, random_state=42, shuffle=False)


# x_train
# y_train
# x_train[-1], y_train[-2]

In [ ]:
x_test[1], y_test[0]

In [ ]:
x_test[-1], y_test[-1]

In [ ]:
x_test = np.array(x_test)
x_train, y_train = np.array(x_train), np.array(y_train)

# Reshape the data
# LSTM expects the input to be 3D
# x_train.shape[0] is the number of samples
# x_train.shape[1] is the number of time steps
# 1 is the number of features

# The reason we need to reshape the data is because LSTM expects the input to be 3D
# In this case, 2332, 60, 1
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

# x_train

In [ ]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(x_train, y_train, batch_size=1, epochs=2)

In [ ]:
# Get the models predicted price values 
predictions_scaled = model.predict(x_test)
predictions = scaler.inverse_transform(predictions_scaled)

# Get the root mean squared error (RMSE)
rmse = np.sqrt(np.mean(((predictions_scaled - y_test) ** 2)))
rmse

In [ ]:
a = scaler.inverse_transform(y_train.reshape(-1, 1)).flatten().tolist()
b = scaler.inverse_transform(np.reshape(y_test,  (1, len(y_test))))[0].tolist()
pred = scaler.inverse_transform(predictions_scaled).flatten().tolist()
# pred

In [ ]:
rmse = np.sqrt(np.mean(((predictions - b) ** 2)))
rmse

In [ ]:
new_table = {'Date': data.iloc[60:, :].index.values,
             'Values': a + b,
             'Prediction': a + pred,
             }

df2 = pd.DataFrame(new_table)

In [ ]:
df2["Date"] = pd.to_datetime(df2["Date"])
df3 = df2.set_index('Date').copy(deep=True)
df3

In [ ]:
# Visualize the data
plt.figure(figsize=(16,6))
plt.title('Model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.plot(df3.tail(200)[['Values', 'Prediction']])
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
plt.show()